# Bloonet Jupyter Notebook

## Required imports

In [ ]:
import requests
import uuid
import json
import time
import pandas as pd
import io
import ipywidgets as widgets
from IPython.display import display
from typing import Union

## Required constants

In [ ]:
SESSION_UUID = uuid.uuid4()
SIMULATION_UUID = uuid.uuid4()
BWS_API_URL = "https://api.bloonetws.siradel.com"
BWS_DL_URL = "https://dl.bloonetws.siradel.com"
RECEPTION_HEIGHTS = [2]
RESOLUTION = 10
ACCESS_TOKEN = None
REFRESH_TOKEN = None
AUTHENTICATION = {
    "url": "https://keycloak.bloonetws.siradel.com/realms/volcanoweb/protocol/openid-connect/token",
    "clientId": "volcano-web-cli",
    "username": "glemoine",
    "password": "xOiKNP73dLaGDSyHDg6q"
}

## Required functions

In [ ]:
def call_request(method: str, url: str, authentication_data: dict,
                 files: list = None, json_content: Union[dict, list] = None,
                 params: dict = None, retry: int = 0, timeout=None) -> requests.Response:
    """
    @summary: call requests with args for use authenticate
    @param method: {str} method request
    @param url: {str} url request
    @param authentication_data: {dict} authentication data
    @param files: {list} files request
    @param json_content: {dict} json request
    @param params: {dict} params request
    @param retry: {int} number of retry
    @param timeout: {Any} timeout request
    @return: {request.Response} response of request
    """

    headers = None

    if authentication_data is None:
        print("Authentication data needed")
    token = get_access_token(authentication_data)
    headers = {"Authorization": f"Bearer {token}"}

    res = None
    if method.upper() == "POST":
        res = requests.post(url=url, files=files, json=json_content, params=params,
                            timeout=timeout, headers=headers, verify=False)
    elif method.upper() == "PUT":
        res = requests.put(url=url, files=files, json=json_content, params=params,
                           timeout=timeout, headers=headers, verify=False)
    elif method.upper() == "GET":
        res = requests.get(url=url, files=files, json=json_content, params=params,
                           timeout=timeout, headers=headers, verify=False)
    elif method.upper() == "DELETE":
        res = requests.delete(url=url, files=files, json=json_content, params=params,
                              timeout=timeout, headers=headers, verify=False)
    else:
        raise ValueError("Method name %S not implemented", method.upper())

    # If access_token expires, refresh the token one time and recall requests
    if res.status_code == 403 and retry == 0:
        refresh_token(authentication_data)
        res = call_request(
            method,
            url,
            authentication_data,
            files,
            json_content,
            params,
            1,
            timeout
        )

    return res


def get_access_token(authentication_data: dict) -> str:
    """
    @summary: Retrieve an access token
    @param authentication_data: {dict} authentication data
    @return: {str} access token provided
    """

    global ACCESS_TOKEN, REFRESH_TOKEN
    if ACCESS_TOKEN is not None:
        return ACCESS_TOKEN
    payload = {
        "grant_type": "password",
        "client_id": authentication_data["clientId"],
        "username": authentication_data["username"],
        "password": authentication_data["password"]
    }

    response = requests.post(
        authentication_data["url"],
        data=payload,
        verify=False
    )

    json_response = response.json()

    if response.status_code == 200:
        ACCESS_TOKEN = json_response["access_token"]
        REFRESH_TOKEN = json_response["refresh_token"]
    else:
        raise RuntimeError("Error call get access token: %s", json_response["error_description"])
    return ACCESS_TOKEN


def refresh_token(authentication_data: dict) -> None:
    """
    @summary: Refresh the access token
    @param authentication_data: {dict} authentication data
    """

    global ACCESS_TOKEN, REFRESH_TOKEN
    if REFRESH_TOKEN is None:
        raise RuntimeError("Error call refresh token : refresh token is empty")

    payload = {
        "grant_type": "refresh_token",
        "client_id": authentication_data["clientId"],
        "refresh_token": REFRESH_TOKEN
    }

    response = requests.post(
        authentication_data["url"],
        data=payload,
        verify=False
    )

    json_response = response.json()

    if response.status_code == 200:
        ACCESS_TOKEN = json_response["access_token"]
        REFRESH_TOKEN = json_response["refresh_token"]
    else:
        raise RuntimeError("Error call get refresh token: %s", json_response["error_description"])


def get_public_models() -> list:
    """
    @summary: Retrieve the public models
    @return: {list} list of public models
    """
    response = call_request(
        method="GET",
        url=f"{BWS_API_URL}/propagationmodels",
        authentication_data=AUTHENTICATION
    )
    public_models_list = response.json()

    if response.status_code != 200:
        raise RuntimeError("Error call get public models: %s", public_models_list["error_description"])

    PUBLIC_MODELS_NAME = ["fixed wireless access", "mobility"]
    public_models = [model for model in public_models_list if model["name"].lower() in PUBLIC_MODELS_NAME and model["type"] is None]
    return public_models


def get_public_antennas() -> list:
    """
    @summary: Retrieve the public antennas
    @return: {list} list of public antennas
    """
    response = call_request(
        method="GET",
        url=f"{BWS_API_URL}/antennas",
        authentication_data=AUTHENTICATION
    )
    public_antennas_list = response.json()

    if response.status_code != 200:
        raise RuntimeError("Error call get public antennas: %s", public_antennas_list["error_description"])

    PUBLIC_ANTENNAS_NAME = [
        "tarana-bn-3ghz-compact-r0",
        "tarana-bn-5ghz-r1",
        "tarana-bn-6ghz-r2",
        "tarana-bn-6ghz-x2-r2"
    ]
    public_antennas = [antenna for antenna in public_antennas_list if antenna["name"].lower() in PUBLIC_ANTENNAS_NAME]
    return public_antennas


## UI for user inputs

In [ ]:
# Widgets upload
label_tx = widgets.Label("Select the CSV file for the transmitters:")
upload_tx = widgets.FileUpload(accept=".csv", multiple=False)

label_rx = widgets.Label("Select the CSV file for the receivers:")
upload_rx = widgets.FileUpload(accept=".csv", multiple=False)

# Widgets dropdown
label_model = widgets.Label("Select the propagation model to use:")
dropdown_model = widgets.Dropdown(
    options=[(m["name"], m["uuid"]) for m in get_public_models()]
)

label_antenna = widgets.Label("Select the antenna diagram to use:")
dropdown_antenna = widgets.Dropdown(
    options=[(m["name"], m["uuid"]) for m in get_public_antennas()]
)

# Display areas
out_tx = widgets.Output()
out_rx = widgets.Output()
out_model = widgets.Output()
out_antenna = widgets.Output()
out_status = widgets.Output()

# Button to start the calculation
btn_run = widgets.Button(
    description="Start the calculation",
    button_style="primary",
    disabled=True
)

# Layout
ui = widgets.VBox([
    label_tx, upload_tx, out_tx,
    widgets.HTML("<hr>"),
    label_rx, upload_rx, out_rx,
    widgets.HTML("<hr>"),
    label_model, dropdown_model, out_model,
    widgets.HTML("<hr>"),
    label_antenna, dropdown_antenna, out_antenna,
    widgets.HTML("<hr>"),
    btn_run,
    out_status
])

display(ui)

# Global variables for storing DataFrames
df_tx = None
df_rx = None


## UI callback functions

In [ ]:
def handle_upload_tx(change):
    global df_tx
    out_tx.clear_output()

    if not upload_tx.value:
        with out_tx:
            print("No transmitters files")
        df_tx = None
        update_button_state()
        return

    # Content recovery
    files = upload_tx.value
    file_info = files[0]
    filename = file_info["name"]
    content = file_info["content"]

    try:
        df_tx = pd.read_csv(io.BytesIO(content), sep=";")
        with out_tx:
            print(f"Transmitters file loaded: {filename}")
            display(df_tx.head())
    except Exception as e:
        df_tx = None
        with out_tx:
            print(f"Error reading CSV transmitters: {e}")

    update_button_state()


def handle_upload_rx(change):
    global df_rx
    out_rx.clear_output()

    if not upload_rx.value:
        with out_rx:
            print("No receivers file")
        df_rx = None
        update_button_state()
        return

    files = upload_rx.value
    file_info = files[0]
    filename = file_info["name"]
    content = file_info["content"]

    try:
        df_rx = pd.read_csv(io.BytesIO(content), sep=";")
        with out_rx:
            print(f"Receivers file loaded: {filename}")
            display(df_rx.head())
    except Exception as e:
        df_rx = None
        with out_rx:
            print(f"Error reading CSV receivers: {e}")

    update_button_state()


def update_button_state():
    btn_run.disabled = not (
        isinstance(df_tx, pd.DataFrame) and 
        isinstance(df_rx, pd.DataFrame) and 
        dropdown_model.value and 
        dropdown_antenna.value
    )


# Connect callbacks to widgets
upload_tx.observe(handle_upload_tx, names="value")
upload_rx.observe(handle_upload_rx, names="value")


## API calls functions

In [ ]:
def create_session() -> dict:
    # Session information
    session = {
        "uuid": str(SESSION_UUID),
        "name": "session_notebook_mp2mp",
        "description": "Session sample MP to MP from Jupyter Notebook"
    }

    # Post session to BWS server
    response = call_request(
        method="POST",
        url=f"{BWS_API_URL}/sessions",
        authentication_data=AUTHENTICATION,
        json_content=session
    )
    result = response.json()

    if response.status_code != 201:
        raise RuntimeError("Error call create session: %s", result["error_description"])
    
    return result


def create_simulation(network_list: pd.DataFrame, antenna_uuid: str, model_uuid: str) -> list:
    # Create simulation object
    propagation_list = []
    for _, network in network_list.iterrows():

        propagation_list.append({
            "baseStation": {
                "x": float(network["transmitter longitude"]),
                "y": float(network["transmitter latitude"]),
                "z": float(network["transmitter height"]),
                "epsgCode": 4326,
                "zmeaning": "ZMEANING_GROUND",
                "azimuth": float(network["azimuth"]),
                "downtilt": float(network["downtilt"]),
                "carrierFrequency": float(network["frequency"]),
                "description": "",
                "sessionUuid": str(SESSION_UUID),
                "name": network["transmitter name"],
                "networkId": network["transmitter id"],
                "transmitPower": float(network["emitting power"]),
                "antennaUuid": antenna_uuid
            },
            "userEquipments": [{
                "sessionUuid": str(SESSION_UUID),
                "description": "",
                "zmeaning": "ZMEANING_GROUND",
                "type": "POINT",
                "name": network["receiver name"],
                "heights": [network["receiver height"]],
                "coordinates": {
                    "x": network["receiver longitude"],
                    "y": network["receiver latitude"],
                    "epsgCode": 4326
                }
            }],
            "propagationModelUuid": model_uuid
        })

        
    simulation_request = {
        "uuid": str(SIMULATION_UUID),
        "name": "session_sample_mp2mp_notebook_simulation",
        "calculationSessionUuid": str(SESSION_UUID),
        "propagationRequest": {
            "propagationScenarios": propagation_list,
            "resultTypes": ["RECEIVED_POWER"],
            "heights": RECEPTION_HEIGHTS,
            "zmeaning": "ZMEANING_GROUND"
        }
    }

    multipart_form_data = [("json", (None, json.dumps(simulation_request), "application/json")), *[]]

    # Post simulation to BWS server
    response = call_request(
        method="POST",
        url=f"{BWS_API_URL}/simulations",
        authentication_data=AUTHENTICATION,
        files=multipart_form_data
    )
    simulation = response.json()

    if response.status_code != 201:
        raise RuntimeError("Error call create session: %s", simulation["error_description"])
    
    return simulation


def wait_for_simulation():

    while True:
        time.sleep(5)
        response = call_request(
            method="GET",
            url=f"{BWS_API_URL}/simulations/{str(SIMULATION_UUID)}/status",
            authentication_data=AUTHENTICATION
        ).json()
        
        with out_status:
            print(response["progress"])
            
            if response["state"] == "DONE":
                print(f"Simulation {str(SIMULATION_UUID)} finished with state {response['state']}")
                break
            if response["state"] == "ERROR":
                print(f"Simulation {str(SIMULATION_UUID)} finished with state {response['state']}")
                print(f"with message {response['error']}")
                print(response['errorMessages'])
                break


def download_simulation_results():
    # Retrieve the simulation results
    res = call_request(
        method="GET",
        url=f"{BWS_API_URL}/simulations/{str(SIMULATION_UUID)}/results",
        authentication_data=AUTHENTICATION
    )

    results = res.json()
    with out_status:
        print(results)



## Calculation actions

In [ ]:
def on_run_clicked(b):
    out_status.clear_output()

    if df_tx is None or df_rx is None:
        with out_status:
            print("Both files must be loaded correctly")
        return
    
    if dropdown_model.value is None:
        with out_status:
            print("A model must be selected")
        return
    
    if dropdown_antenna.value is None:
        with out_status:
            print("An antenna must be selected")
        return
    
    df_network = df_tx.merge(df_rx, how="cross")
    with out_status:
        display(df_network.head())

    create_session()
    create_simulation(df_network, dropdown_antenna.value, dropdown_model.value)
    wait_for_simulation()
    download_simulation_results()

btn_run.on_click(on_run_clicked)
